# Step 1. Parameter Setting 

In [1]:
import dtnmonitor_v2 as dtn
import subprocess,datetime,sys

def exec_print(command):
  with open(logfile, 'a') as f:
      process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
      f.write("\n# command: " + command + "(time:" + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + ")\n")
      for line in iter(process.stdout.readline, b''):
          # system print
          sys.stdout.write(line)
          # log
          f.write(line.decode(sys.stdout.encoding) )


logfile='/tmp/test2.log' # log file

graph_mode=1  #0 all, 1 100pt



# Step 2. 100G DTN System Checking Instruction Guild

## 2.1. To change TCP parameters, edit /etc/sysctl.conf as:

In [2]:
# net.core.default_qdisc = fq
# net.core.netdev_max_backlog=250000
# net.ipv4.tcp_no_metrics_save=1
# net.ipv4.tcp_congestion_control=htcp
# net.ipv4.tcp_mtu_probing=1
# net.core.rmem_max=671088640
# net.core.wmem_max=671088640
# net.core.rmem_default=335544320
# net.core.wmem_default=335544320
# net.core.optmem_max=335544320
# net.ipv4.tcp_mem=33554432 335544320 2355443200
# net.ipv4.tcp_rmem = 409600 100873800 2147483647
# net.ipv4.tcp_wmem = 409600 100873800 2147483647

In [3]:
# check it
# show your system config
#dtn.exec_sysout("sysctl -a |grep 'net.core\|net.ipv4.tcp'")
#dtn.exec_print("sysctl -a |grep 'net.core\|net.ipv4.tcp'")
exec_print("sudo sysctl -a |grep 'net.core\|net.ipv4.tcp'")

net.core.bpf_jit_enable = 0
net.core.busy_poll = 0
net.core.busy_read = 0
net.core.default_qdisc = fq
net.core.dev_weight = 64
net.core.message_burst = 10
net.core.message_cost = 5
net.core.netdev_budget = 300
net.core.netdev_max_backlog = 250000
net.core.netdev_rss_key = 8e:8b:a2:bc:dc:7e:0d:e2:99:33:f9:97:26:42:ae:01:f9:d7:f2:3b:5a:cc:d0:83:13:a4:86:ee:41:ae:c0:34:de:fa:c2:7c:12:bf:1b:e6:62:f1:85:9a:fd:14:69:5c:64:63:1c:5f
net.core.netdev_tstamp_prequeue = 1
net.core.optmem_max = 335544320
net.core.rmem_default = 335544320
net.core.rmem_max = 671088640
net.core.rps_sock_flow_entries = 0
net.core.somaxconn = 128
net.core.warnings = 1
net.core.wmem_default = 335544320
net.core.wmem_max = 671088640
net.core.xfrm_acq_expires = 30
net.core.xfrm_aevent_etime = 10
net.core.xfrm_aevent_rseqth = 2
net.core.xfrm_larval_drop = 1
net.ipv4.tcp_abort_on_overflow = 0
net.ipv4.tcp_adv_win_scale = 1
net.ipv4.tcp_allowed_congestion_control = htcp cubic reno
net.ipv4.tcp_app_win = 31
net.ipv4.tcp_autoc

## 2.2 update Mellanox driver to the most recent one (4.1) and tuning script package.


In [ ]:
### Run the tuning script in priority: 
# sudo mlnx_tune


In [4]:
# check it
#exec_print('if [ $(ls /tmp/mlnx_tune_*.log) ];then echo "ok, good";else echo "No, you did not do it"; fi;')
exec_print("sudo /usr/sbin/mlnx_tune")

2017-09-06 17:59:48,620 INFO Collecting node information
2017-09-06 17:59:48,620 INFO Collecting OS information
2017-09-06 17:59:48,623 INFO Collecting cpupower information
2017-09-06 17:59:48,626 INFO Collecting watchdog information
2017-09-06 17:59:48,628 INFO Collecting abrt-ccpp information
2017-09-06 17:59:48,631 INFO Collecting abrtd information
2017-09-06 17:59:48,634 INFO Collecting abrt-oops information
2017-09-06 17:59:48,636 INFO Collecting alsa-state information
2017-09-06 17:59:48,638 INFO Collecting anacorn information
2017-09-06 17:59:48,641 INFO Collecting atd information
2017-09-06 17:59:48,644 INFO Collecting avahi-daemon information
2017-09-06 17:59:48,646 INFO Collecting bluetooth information
2017-09-06 17:59:48,649 INFO Collecting certmonger information
2017-09-06 17:59:48,651 INFO Collecting cups information
2017-09-06 17:59:48,653 INFO Collecting halddaemon information
2017-09-06 17:59:48,656 INFO Collecting hidd information
2017-09-06 17:59:48,658 INFO Collectin

## 2.3 Bind NIC interrupt with the correct numa cores using:


In [ ]:
### /user/sbin/set_irq_affinity_bynode.sh <numa_node> <interface>


In [ ]:
# no check command

## 2.4 Check PCIe Width, Speed, Max Payload Size and Max Read Request

In [ ]:
#lspci: https://community.mellanox.com/docs/DOC-2496
# /usr/bin/setpci -s 41:00.0 68.W=5000
### Especially set Max Read Request to 4096

In [ ]:

exec_print('if [ $(lspci -s 04:00.0 -vvv | grep MaxReadReq) ];then echo "ok, good";else echo "No, you did not do it"; fi;')


## 2.5. Increase txqueuelen to 10000:


In [ ]:
### ifconfig <interface> txqueuelen 10000

In [ ]:
exec_print('ifconfig |grep txqueuelen')

## 2.6 Set CPU gonvener setting to performance:

In [ ]:
### cpupower frequency-set -g performance

In [ ]:
exec_print('cpupower frequency-info -g')

## 2.7. Turn off irqbalance and tuned:

In [ ]:
### service  irqbalance stop
### service  tuned stop

In [ ]:
exec_print('service irqbalance status ')

In [ ]:
exec_print('service tuned status ')

# Step 3. Just monitor

In [ ]:
## download and monitor
dtn.exec_command("python callmonitor_v2.py",graph_mode)

## you can stop monitoring anytime when you press "interrupt kernel"